In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import fast_dsift, no_op, hog, dsift

from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images

from menpofit.aam import (
    AAMBuilder, PatchAAMBuilder, LinearAAMBuilder, LinearPatchAAMBuilder, PartsAAMBuilder,
    LucasKanadeAAMFitter,
    WibergInverseCompositional, 
    ProjectOutInverseCompositional,
    AlternatingInverseCompositional,
    ModifiedAlternatingInverseCompositional,
    SimultaneousInverseCompositional,
    holistic_sampling_from_step)
from menpofit.visualize import visualize_fitting_result, plot_ced

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=25):
    i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    training_images.append(i)

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=25):    
    i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    if i.n_channels == 3:
        i = i.as_greyscale(mode='average')
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Build Active Appearance Models

In [ ]:
features = fast_dsift
diagonal = 100
scales = (1, .5)
scale_shapes = False
scale_features = True
max_shape_components= 25
max_appearance_components = 200

#### AAM

In [ ]:
aam_builder = AAMBuilder(
    features=features,
    diagonal=diagonal, 
    scales=scales, 
    scale_shapes=scale_shapes,
    scale_features=scale_features,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

aam_batch = aam_builder.build(
    training_images, 
    group='ibug_face_66', 
    verbose=True)

In [ ]:
aam_incremental = aam_builder.build_incrementally(
    training_images, 
    group='ibug_face_66',
    batch_size=500,
    verbose=True)

In [ ]:
aam_batch.appearance_models[0].mean().view()

In [ ]:
aam_incremental.appearance_models[0].mean().view()

## Fit Active Appearance Models

#### AAM Fitters at different sampling rates

In [ ]:
aam = aam_batch

In [ ]:
lk_algorithm_cls = WibergInverseCompositional
n_shape = [5, 15]
n_appearance = [50, 100]

In [ ]:
sampling_2, mask_2 = holistic_sampling_from_step(aam, step=2)
sampling_4, mask_4 = holistic_sampling_from_step(aam, step=4)
sampling_8, mask_8 = holistic_sampling_from_step(aam, step=8)

In [ ]:
mask_2.view()

In [ ]:
mask_4.view()

In [ ]:
mask_8.view()

In [ ]:
aam_fitter_1 = LucasKanadeAAMFitter(
    aam, 
    lk_algorithm_cls=lk_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance)

In [ ]:
aam_fitter_2 = LucasKanadeAAMFitter(
    aam, 
    lk_algorithm_cls=lk_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    sampling=sampling_2)

In [ ]:
aam_fitter_4 = LucasKanadeAAMFitter(
    aam, 
    lk_algorithm_cls=lk_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    sampling=sampling_4)

In [ ]:
aam_fitter_8 = LucasKanadeAAMFitter(
    aam, 
    lk_algorithm_cls=lk_algorithm_cls, 
    n_shape=n_shape, 
    n_appearance=n_appearance,
    sampling=sampling_8)

#### Fitting loop

In [ ]:
fitters = [aam_fitter_1,
           aam_fitter_2,
           aam_fitter_4,
           aam_fitter_8]
           
fitters_results = []

for j, i in enumerate(test_images[:]):
    
    gt_s = i.landmarks['ibug_face_66'].lms
    np.random.seed(2)
    s = fitters[0].noisy_shape_from_shape(gt_s, noise_std=0.0)

    for fitter in fitters:
        
        fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=20) 
        fr.downscale = 0.5
        fitters_results.append(fr)
    
        print 'Image: ', j
        print fr

##  Results

In [ ]:
initial_errors = [fr.initial_error() for fr in fitters_results[::4]]

final_errors_1 = [fr.final_error() for fr in fitters_results[::4]]
final_errors_2 = [fr.final_error() for fr in fitters_results[1::4]]
final_errors_4 = [fr.final_error() for fr in fitters_results[2::4]]
final_errors_8 = [fr.final_error() for fr in fitters_results[3::4]]

errors = [initial_errors, final_errors_1, final_errors_2, final_errors_4, final_errors_8]

In [ ]:
plot_ced(errors, legend_entries=['ini', '1', '2', '4', '8'])

In [ ]:
print 'ini:', np.mean(initial_errors), 'std_1:', np.std(initial_errors), 'median:', np.median(initial_errors) 
print 'mean_1:', np.mean(final_errors_1), 'std_1:', np.std(final_errors_1), 'median:', np.median(final_errors_1) 
print 'mean_2:', np.mean(final_errors_2), 'std_2:', np.std(final_errors_2), 'median:', np.median(final_errors_2) 
print 'mean_4:', np.mean(final_errors_4), 'std_4:', np.std(final_errors_4), 'median:', np.median(final_errors_4) 
print 'mean_8:', np.mean(final_errors_8), 'std_8:', np.std(final_errors_8), 'median:', np.median(final_errors_8) 

In [ ]:
from scipy import stats  

# ANOVA test
f_val, p_val1 = stats.f_oneway(final_errors_1, final_errors_2, 
                              final_errors_4, final_errors_8)  

# Kruskal-Wallis H-test
h_stat, p_val2 = stats.kruskal(final_errors_1, final_errors_2, 
                             final_errors_4, final_errors_8)

print "One-way ANOVA,", "\tp-value:", p_val1, "f-value:", f_val
print "Kruskal-Wallis," "\tp-value:", p_val2, "h-stat:", h_stat  

In [ ]:
visualize_fitting_result(fitters_results[::4])

In [ ]:
visualize_fitting_result(fitters_results[1::4])

In [ ]:
visualize_fitting_result(fitters_results[2::4])

In [ ]:
visualize_fitting_result(fitters_results[3::4])

In [ ]:
%timeit -n 25 fr = aam_fitter_1.fit(i, s, gt_shape=gt_s, max_iters=20) 
%timeit -n 25 fr = aam_fitter_2.fit(i, s, gt_shape=gt_s, max_iters=20) 
%timeit -n 25 fr = aam_fitter_4.fit(i, s, gt_shape=gt_s, max_iters=20) 
%timeit -n 25 fr = aam_fitter_8.fit(i, s, gt_shape=gt_s, max_iters=20) 